#*FOR BODY DAMAGE*

#**IMAGE PRE PROCESSING**

**1. Import The ImageDataGenerator Library**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

**2. Configure ImageDataGenerator Class**

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.1,
                                   zoom_range = 0.1,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

**3. Apply ImageDataGenerator Functionality To Trainset And Testset**

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Car_damage/body/training',
                                                 target_size=(224, 224),
                                                 batch_size=10,
                                                 class_mode='categorical')
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Car_damage/body/validation',
                                            target_size=(224, 224),
                                            batch_size=10,
                                            class_mode='categorical')

Found 979 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


#**MODEL BUILDING**

**1. Importing The Model Building Libraries**

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

**2. Loading The Model**

In [ ]:
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/Car_damage/body/training'
valid_path = '/content/drive/MyDrive/Car_damage/body/validation'

In [ ]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


**3. Adding Flatten Layer**

In [ ]:
for layer in vgg16.layers:
    layer.trainable = False

In [ ]:
folders = glob('/content/drive/MyDrive/Car_damage/body/training/*')

In [ ]:
folders

['/content/drive/MyDrive/Car_damage/body/training/02-side',
 '/content/drive/MyDrive/Car_damage/body/training/01-rear',
 '/content/drive/MyDrive/Car_damage/body/training/00-front']

In [ ]:
x = Flatten()(vgg16.output)

In [ ]:
len(folders)

3

**4. Adding Output Layer**

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

**5. Creating A Model Object**

In [ ]:
model = Model(inputs=vgg16.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

**6. Configure The Learning Process**

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

**7. Train The Model**

In [17]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=25,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/25
98/98 [==============================] - 498s 5s/step - loss: 1.3552 - accuracy: 0.5352 - val_loss: 1.2528 - val_accuracy: 0.5906
Epoch 2/25
98/98 [==============================] - 421s 4s/step - loss: 0.6244 - accuracy: 0.7600 - val_loss: 0.8677 - val_accuracy: 0.6725
Epoch 3/25
98/98 [==============================] - 422s 4s/step - loss: 0.4846 - accuracy: 0.8161 - val_loss: 0.8827 - val_accuracy: 0.6784
Epoch 4/25
98/98 [==============================] - 420s 4s/step - loss: 0.3649 - accuracy: 0.8682 - val_loss: 1.0304 - val_accuracy: 0.6199
Epoch 5/25
98/98 [==============================] - 417s 4s/step - loss: 0.2899 - accuracy: 0.8968 - val_loss: 0.9003 - val_accuracy: 0.6316
Epoch 6/25
98/98 [==============================] - 418s 4s/step - loss: 0.3088 - accuracy: 0.8866 - val_loss: 1.0522 - val_accuracy: 0.6667
Epoch 7/25
98/98 [==============================] - 420s 4s/step - loss: 0.1704 - accuracy: 0.9397 - val_loss: 0.9331 - val_accuracy: 0.7018
Epoch 8/25
98

**8. Save The Model**

In [18]:
from tensorflow.keras.models import load_model

model.save('/content/drive/MyDrive/IBM/Model/body.h5')

**9. Test The Model**

In [21]:
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize

In [22]:
model = load_model('/content/drive/MyDrive/IBM/Model/body.h5')

In [23]:
def detect(frame):
  img = cv2.resize(frame,(224,224))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

  if(np.max(img)>1):
    img = img/255.0
  img = np.array([img])
  prediction = model.predict(img)
  label = ["front","rear","side"]
  preds = label[np.argmax(prediction)]
  return preds

In [24]:
import numpy as np

In [25]:
data = "/content/drive/MyDrive/Car_damage/level/training/01-minor/0008.JPEG"
image = cv2.imread(data)
print(detect(image))

1/1 [==============================] - 1s 536ms/step
side


#*FOR LEVEL* *DAMAGE*

#**IMAGE PRE PROCESSING**

**1. Import The ImageDataGenerator Library**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

**2. Configure ImageDataGenerator Class**

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.1,
                                   zoom_range = 0.1,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

**3. Apply ImageDataGenerator Functionality To Trainset And Testset**

In [26]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Car_damage/level/training',
                                                 target_size = (224, 224),
                                                 batch_size = 10,
                                                 class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Car_damage/level/training',
                                            target_size = (224, 224),
                                            batch_size = 10,
                                            class_mode = 'categorical')

Found 979 images belonging to 3 classes.
Found 979 images belonging to 3 classes.


#**MODEL BUILDING**

**1. Importing The Model Building Libraries**

In [27]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

**2. Loading The Model**

In [28]:
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/Car_damage/level/training'
valid_path = '/content/drive/MyDrive/Car_damage/level/training'

In [29]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

**3. Adding Flatten Layer**

In [30]:
for layer in vgg16.layers:
    layer.trainable = False

In [32]:
folders = glob('/content/drive/MyDrive/Car_damage/level/training/*')

In [31]:
folders

['/content/drive/MyDrive/Car_damage/body/training/02-side',
 '/content/drive/MyDrive/Car_damage/body/training/01-rear',
 '/content/drive/MyDrive/Car_damage/body/training/00-front']

In [33]:
x = Flatten()(vgg16.output)

In [34]:
len(folders)

3

**4. Adding Output Layer**

In [35]:
prediction = Dense(len(folders), activation='softmax')(x)

**5. Creating A Model Object**

In [36]:
model = Model(inputs=vgg16.input, outputs=prediction)

In [37]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

**6. Configure The Learning Process**

In [38]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

**7. Train The Model**

In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=25,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/25
98/98 [==============================] - 615s 6s/step - loss: 1.2465 - accuracy: 0.5516 - val_loss: 1.0659 - val_accuracy: 0.5731
Epoch 2/25
98/98 [==============================] - 604s 6s/step - loss: 0.6654 - accuracy: 0.7549 - val_loss: 1.0368 - val_accuracy: 0.6316
Epoch 3/25
98/98 [==============================] - 604s 6s/step - loss: 0.5950 - accuracy: 0.7630 - val_loss: 1.1309 - val_accuracy: 0.6257
Epoch 4/25
98/98 [==============================] - 601s 6s/step - loss: 0.4964 - accuracy: 0.8069 - val_loss: 1.1262 - val_accuracy: 0.6082
Epoch 5/25
98/98 [==============================] - 603s 6s/step - loss: 0.3559 - accuracy: 0.8672 - val_loss: 1.1408 - val_accuracy: 0.6316
Epoch 6/25
98/98 [==============================] - 604s 6s/step - loss: 0.2425 - accuracy: 0.9152 - val_loss: 1.1566 - val_accuracy: 0.5789
Epoch 7/25
98/98 [==============================] - 604s 6s/step - loss: 0.1964 - accuracy: 0.9367 - val_loss: 1.1200 - val_accuracy: 0.6199
Epoch 8/25
98

**8. Save The Model**

In [39]:
from tensorflow.keras.models import load_model

model.save('/content/drive/MyDrive/IBM/Model/level.h5')

**9. Test The Model**

In [40]:
from tensorflow.keras.models import load_model
import cv2
from skimage.transform import resize

In [41]:
model = load_model('/content/drive/MyDrive/IBM/Model/level.h5')

In [42]:
def detect(frame):
  img = cv2.resize(frame,(224,224))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

  if(np.max(img)>1):
    img = img/255.0
  img = np.array([img])
  prediction = model.predict(img)
  label = ["minor","moderate","severe"]
  preds = label[np.argmax(prediction)]
  return preds

In [43]:
import numpy as np

In [44]:
data = "/content/drive/MyDrive/Car_damage/level/validation/01-minor/0002.JPEG"
image = cv2.imread(data)
print(detect(image))

1/1 [==============================] - 1s 510ms/step
severe
